In [1]:
import pandas as pd

In [2]:
csv_file = "data/posts/etiquetado.csv"
df_posts = pd.read_csv(csv_file)
df_posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   titulo       124 non-null    object
 1   cuerpo       124 non-null    object
 2   sentimiento  124 non-null    object
dtypes: object(3)
memory usage: 3.0+ KB


In [3]:
csv_file = "data/comentarios/etiquetado.csv"
df_comments = pd.read_csv(csv_file)
df_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   208 non-null    int64 
 1   titulo       208 non-null    object
 2   cuerpo       208 non-null    object
 3   sentimiento  208 non-null    object
dtypes: int64(1), object(3)
memory usage: 6.6+ KB


In [4]:
df_comments = df_comments.drop(columns="Unnamed: 0")

In [5]:
df_comments["texto"] = df_comments["cuerpo"].fillna("")
df_posts["texto"] = df_posts["titulo"].fillna("") + " " + df_posts["cuerpo"].fillna("")

In [6]:
# Unir los dataframes de posts y comentarios
df = pd.concat([df_posts, df_comments], ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332 entries, 0 to 331
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   titulo       332 non-null    object
 1   cuerpo       332 non-null    object
 2   sentimiento  332 non-null    object
 3   texto        332 non-null    object
dtypes: object(4)
memory usage: 10.5+ KB


In [7]:
df["sentimiento"].value_counts()

sentimiento
NEUTRAL      189
NEGATIVO     103
POSITIVO      39
 NEGATIVO      1
Name: count, dtype: int64

In [8]:
# Agregar más comentarios positivos obtenidos de facebook para que las clases no estén tan desbalanceadas
facebook_comments = [
    {
        "titulo": "BCP advierte que criptomonedas no tienen registro ni autorización en Paraguay",
        "cuerpo": """Mejor ahorren en Dólares que se devalúa cada año antes que meter dinero en algo que se valua en promedio anual un 100%
Desde el 2020 hasta ahora hay un 33% de devaluación del dólar (Dicho por Trump en campaña), ni que decir los guaraníes cuánto estarán devaluados mientras te tiran que solo "hay inflación del 4% anual, lo cual sigue siendo un robo)
Mientras que desde el 2020 hasta ahora bitcoin tiene una valuación del 900% promedio (a largo plazo)
Tncs? Compra y guarda con tus propias llaves privadas en wallets de código abierto
No hagan trading o los liquidan (no sean la liquidez de los intercambios haciendo trading apalancándose un x10 para arriba)""",
        "sentimiento": "POSITIVO"
    },
    {
        "titulo": "BCP advierte que criptomonedas no tienen registro ni autorización en Paraguay",
        "cuerpo": """Mientras los políticos ya están comprando, no quieren que la gente compre, o acaso no está el gobierno trabajando ya con Morphware?. Jajajaja payasos, gracias a ustedes la gente es ignorante y vive dormida y en pobreza, imaginate esta publicación, para desanimar a la gente, payasos""",
        "sentimiento": "POSITIVO"
    },
    {
        "titulo": "Las criptomonedas no son registradas ni autorizadas por el Banco Central del Paraguay",
        "cuerpo": """Las criptomonedas no necesitan la autorización de ningún Banco Central, el Bitcoin existe desde el 2009 y va a seguir existiendo.""",
        "sentimiento": "POSITIVO"
    },
    {
        "titulo": "Las criptomonedas no son registradas ni autorizadas por el Banco Central del Paraguay",
        "cuerpo": """Las cripto nacieron para ser libres""",
        "sentimiento": "POSITIVO"
    },
    {
        "titulo": "Criptomonedas: ¿Es rentable el negocio?",
        "cuerpo": """La criptomonedas en buenas manos , te hace ganar mucho dinero ,y lo de la minería es Opcional podés ganar criptomonedas sin hacer minería...pero si querés minar con una máquina para minar no ganas nada , necesitas cientos y miles de maquinas de minería ....El tener criptomonedas es muy interesante ( la moneda principal es Bitcoins) pero existen cientos de monedas nuevas , cada día sale una moneda nueva ...y mucha gente lo que hace es comprar un criptomoneda cuando está barato y lo vende cuando está caro y ahí ganas dinero ...no es una estafa es algo que sí existe ....las estafas es cuando alguien te dice compra esto y no existe lo que te va a hacer comprar o te dice que ganarás mucha plata pero perder tu dinero ....pero como dije la criptomonedas en buenas manos le podés quitar buenas ganancias ...y otra cosa podés tener años y años nunca pierdes nada en el caso que vos mismo quieras efectivizar a guaraní esas criptomonedas en mi caso desde el 2010 que tengo criptomonedas""",
        "sentimiento": "POSITIVO"
    },
    {
        "titulo": "",
        "cuerpo": """Paraguay está quedándose atrás.
Las criptomonedas no son una moda pasajera: son el futuro del sistema financiero global. ITAIPU, con su enorme potencial energético, podría convertirse en uno de los centros de minería más importantes del mundo, generando riqueza e innovación para nuestro país.
Necesitamos políticos jóvenes, preparados, visionarios, que miren hacia adelante, que no lleguen al poder por contactos ni nepotismo, sino por su talento y compromiso con un salto cuántico hacia el progreso.
Hoy, en nuestro Congreso, muchos ni siquiera comprenden cómo funcionan las criptomonedas. Pero no debemos tenerle miedo al cambio. Estados Unidos ya está invirtiendo en este ecosistema. El Salvador ya lo hizo.
¿Vamos a seguir dependiendo eternamente del dólar?
¿Por qué no soñar con tener nuestra propia moneda digital, respaldada por proyectos sólidos, leyes modernas y una estructura financiera renovada?
Si no lo hacemos hoy, la evolución lo hará por nosotros mañana. Y quien lidere este cambio, quien tenga el coraje de adelantarse al movimiento, quedará para siempre en la historia.""",
        "sentimiento": "POSITIVO"
    },
    {
        "titulo": "El aumento de la inversión en criptominería en paraguay explicado por un experto",
        "cuerpo": """La herramienta que va liberar del yugo de los gobiernos y la inflación!""",
        "sentimiento": "POSITIVO"
    },
    {
        "titulo": "El auge del bitcoin en Paraguay: ¿Cómo funciona?",
        "cuerpo": """Soy paraguayo vivo x bueno Aires, agradezco la verdad que conozco otras formas de ver y ganar un poco,, l criptoactivo vino para quedar y será parte de nuestra vida cotidiana, el dinero que conocemos de a poco vana ir quedando atrás, te invito a que empiece a educarse sobre el tema y ser parte de la nueva generación de nuevo forma de pagos digitales,!!""",
        "sentimiento": "POSITIVO"
    },
    {
        "titulo": "El auge del bitcoin en Paraguay: ¿Cómo funciona?",
        "cuerpo": """En otros países hace años que se usa el bitcoin.. acá a penas sale y la gente tavy comentando sin saber, es muchísimo más fácil, sólo que acá como siempre estamos tan atrasados, años luz atrasados. Además de que se gana mucho más con las criptomonedas..""",
        "sentimiento": "POSITIVO"
    },
    {
        "titulo": "El auge del bitcoin en Paraguay: ¿Cómo funciona?",
        "cuerpo": """En líneas generales el Bitcoin viene afianzándose como moneda a varios años después de la caída que tuvo, es un tema interesante para aquellas personas que les gustaría entender sobre el Mercado de divisas, para quien no le gusta sobre este tipo de Mercado es una información más! Más para aquellos que les interesa es una oportunidad de entrar en el mundo digital de las criptomonedas o otro tipo de inversiones, la pandemia mostró que todo se vuelca para lo digital, existen correctoras que hacen contratos con personas de forma de renda pasiva y ellos trabajan tus capitales y te devuelven los intereses de las inversiones en BITCOIN, criticar algo que uno no entiende es fácil con personas que ya a años usan este medio para ganar dinero !""",
        "sentimiento": "POSITIVO"
    },
    {
        "titulo": "El auge del bitcoin en Paraguay: ¿Cómo funciona?",
        "cuerpo": """Bitcoin en si no es estafa, es un sistema de transacción distribuido e insensurable, ningún gobierno puede confiscarte tus activos. Su funcionamiento es en base a matemática pura y tiene un libro contable público donde cualquier usuario puede consultar de dónde viene o va algún activo. Pero siempre hay que cuidarse con los que te ofrecen dinero rápido y ganancias desmesuradas. Siempre antes de invertir es bueno leer y preguntar no confiar en nadie por lo que diga, pero Bitcoin llegó para quedarse y hará los próximos millonarios de este nuevo siglo.""",
        "sentimiento": "POSITIVO"
    },
    {
        "titulo": "El auge del bitcoin en Paraguay: ¿Cómo funciona?",
        "cuerpo": """Lo feliz que me pongo cuando abro mi billetera de Blockchain y de Trust Wallet😍😍😍 pero aún me duele perder 1 BTC en Gladiacoin....pero no importa fue mi aprendizaje.... Pero más me duele no haber comprado más.... Las criptomonedas son el futuro y las que ingresan ahora no se van a arrepentir jamás""",
        "sentimiento": "POSITIVO"
    },
    {
        "titulo": "El auge del bitcoin en Paraguay: ¿Cómo funciona?",
        "cuerpo": """Si hubiera comprado comprado
Solo cuando estaba 200 usd
En estos momentos ya seria millonario!!🤣🤣
Hoy en dia esta cotizando a 34000 usd🤑🤑🤑🤑
""",
        "sentimiento": "POSITIVO"
    }
]

df_facebook = pd.DataFrame(facebook_comments)
df_facebook["texto"] = df_facebook["cuerpo"].fillna("")

In [9]:
# Agregar más comentarios positivos obtenidos de twitter/x para que las clases no estén tan desbalanceadas
x_comments = [
{
"titulo": "Bitcoin gana terreno en Paraguay",
"cuerpo": "Inversionistas en Bitcoin también están encontrando su lugar en Paraguay. Con una creciente comunidad cripto, algunos incluso pagan sus trámites en BTC. Paraguay se perfila como un destino atractivo para la economía digital.",
"sentimiento": "POSITIVO"
},
{
"titulo": "Paraguay como HUB cripto",
"cuerpo": "Se viene Paraguay como HUB cripto en temas de compliance, servicios para las fuerzas de seguridad como así también, exchanges, bancos inversores grandes y chicos. Paraguay será líder en temas cripto para exportar servicios a todo el mundo.",
"sentimiento": "POSITIVO"
},
{
"titulo": "Centro de inteligencia cripto",
"cuerpo": "Después de un año de arduo trabajo, nos complace anunciar que Paraguay será la sede del primer centro de inteligencia cripto de la región. Este centro será un recurso invaluable para las empresas y los gobiernos que buscan comprender y aprovechar el potencial de las criptomonedas.",
"sentimiento": "POSITIVO"
},
{
"titulo": "Potencial del mercado cripto en Paraguay",
"cuerpo": "¿Por qué Criptomonedas en Paraguay? Para comenzar, existe un mercado de US$ 2 billones. Las Criptomonedas tendrán casi 7 millones de nuevos usuarios potenciales y los más de 870.000 compatriotas en el extranjero, una forma más de transferir remesas.",
"sentimiento": "POSITIVO"
},
{
"titulo": "Cripto: riesgos y oportunidades",
"cuerpo": "Cripto tiene sus riesgos, obvio, pero también te da chance de pegar un salto.",
"sentimiento": "POSITIVO"
},
{
"titulo": "Cripto y descentralización",
"cuerpo": "Lo primero que deberían es estudiarlas. Cómo van a estar listadas si son descentralizadas, se crearon para justamente eliminar al tercero de confianza (el problema de los generales bizantinos). Los pagos crossborder hoy son cada vez más con stablecoin, ¿estarán preocupados?",
"sentimiento": "POSITIVO"
},
{
"titulo": "Trazabilidad y legitimidad de cripto",
"cuerpo": "Cripto es más trazable que el dinero físico. Todas las transacciones se registran en un libro contable digital llamado blockchain. Además, Bitcoin es un activo con derivados financieros como futuros en la bolsa de Chicago y ETF en la bolsa de Nueva York.",
"sentimiento": "POSITIVO"
},
{
"titulo": "Ventajas de Bitcoin",
"cuerpo": "A Bitcoin no le pasa esto: funciona 24/7/365 y, con Lightning Network, ofrece pagos instantáneos, costos bajos y transferencias globales sin tasas abusivas ni preguntas innecesarias. ¿Lo han probado?",
"sentimiento": "POSITIVO"
},
{
"titulo": "Comunidad Bitcoin Paraguay",
"cuerpo": "Feliz de anunciar #Bitcoin Paraguay! Una comunidad centrada en la educación y el networking para fomentar el crecimiento del ecosistema en Paraguay. Contamos con el apoyo de diversas iniciativas bitcoiners en todo el mundo. La idea es organizar meetups, talleres y eventos en todo el país.",
"sentimiento": "POSITIVO"
},
{
"titulo": "Bitcoin vs inflación",
"cuerpo": "Dime de nuevo Paraguay, ¿quieres más inflación o ya entiendes que quieres la moneda más estrictamente emitida del mundo, el #bitcoin?",
"sentimiento": "POSITIVO"
},
{
"titulo": "Valor del Bitcoin",
"cuerpo": "El valor de 1 #Bitcoin hoy equivale aproximadamente a 22 años de salario mínimo en Paraguay.",
"sentimiento": "POSITIVO"
},
{
"titulo": "Crecimiento del ecosistema",
"cuerpo": "El ecosistema de Bitcoin crece en Paraguay impulsado por la minería y la educación.",
"sentimiento": "POSITIVO"
},
{
"titulo": "Bitcoin Paraguay: encuentro inspirador",
"cuerpo": "“Fue una conversación inspiradora entre personas con trayectorias, contextos y experiencias distintas, reunidas en Asunción por primera vez con el propósito común de pensar, construir y mejorar Bitcoin”, dijo Juan Pessolani, voluntario de Bitcoin Paraguay.",
"sentimiento": "POSITIVO"
},
{
"titulo": "",
"cuerpo": """Paraguay tiene todo para ser un hub tecnológico en América Latina, según Cofundadora de X4T

📌X4T nació para resolver una necesidad concreta: facilitar el acceso a criptomonedas desde Paraguay. Con una plataforma 100% local, apuesta por la educación financiera y la inclusión digital.

👉La empresa nace a partir de dos necesidades que detectamos: la falta de un exchange local y la necesidad de acercar el mundo cripto a las personas, explicó Nicole Cobelli, Vicepresidente y Cofundadora de la plataforma durante el evento Mujeres Power Summit.""",
"sentimiento": "POSITIVO"
},
{
"titulo": "",
"cuerpo": """¡Qué lindo que va a estar esto 🇵🇾! 

Paraguay Blockchain Summit. 

@criptoboi
 es un gran builder de la región y me alegra ver el crecimiento que tanto el, su equipo y la comunidad paraguaya están logrando para acelerar la adopción de criptomonedas.""",
"sentimiento": "POSITIVO"
},
{
"titulo": "",
"cuerpo": """Desde hace años, en @cripto_py venimos desarrollando el mercado cripto en Paraguay. El Paraguay Blockchain Summit  @PBSpy_
 representa el resultado de años de esfuerzo.

CriptoPy se une a la Cámara Paraguaya de Fintech con una misión clara: convertir a Asunción en la capital blockchain de Latinoamérica.

Regulación, medios de pago, educación y tokenización, todo en un solo lugar.

Speakers internacionales, reguladores locales y comunidades globales reunidos por una sola causa: el futuro descentralizado.""",
"sentimiento": "POSITIVO"
},
{
"titulo": "",
"cuerpo": """🚨 RESUMEN DE LAS ÚLTIMAS 8 HORAS 👇

🌍 Paraguay se está convirtiendo en un centro importante para Bitcoin con grandes inversiones en minería.

💡 Programas educativos como "Mi Primer Bitcoin" impulsan el conocimiento financiero en criptomonedas.

📈 Ventas de XRP en Binance muestran una fuerte demanda, sugiriendo una tendencia alcista próximamente.

🏦 Standard Chartered se une a la ola de adopción de Bitcoin, ofreciendo nuevas opciones de inversión cripto.

🔗 La actualización Pectra impulsa la actividad de Ethereum con mayor uso de aplicaciones descentralizadas.

💶 El Banco Central Europeo explora la implementación del euro digital para mejorar la eficiencia financiera.""",
"sentimiento": "POSITIVO"
},
{
"titulo": "",
"cuerpo": """🪙 El dinero no siempre fue como lo conocemos hoy.

Empezó con trueque, siguió con oro, llegaron los billetes…
Y ahora estamos entrando a una nueva era: el *dinero digital* 💻

Bitcoin no es solo una moda. Es un sistema que:
✅ Se puede intercambiar
✅ Mide valor
✅ Guarda valor en el tiempo
✅ Es seguro
✅ Es global
✅ Y tiene una cantidad limitada (¡no se puede imprimir más!)

En X4T creemos que entender el futuro del dinero es clave para tomar buenas decisiones hoy.
¿Será Bitcoin la próxima evolución? Vos decidís.""",
"sentimiento": "POSITIVO"
}
]
df_x = pd.DataFrame(x_comments)
df_x["texto"] = df_x["cuerpo"].fillna("")

In [10]:
df = pd.concat([df, df_facebook, df_x], ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363 entries, 0 to 362
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   titulo       363 non-null    object
 1   cuerpo       363 non-null    object
 2   sentimiento  363 non-null    object
 3   texto        363 non-null    object
dtypes: object(4)
memory usage: 11.5+ KB


In [11]:
df["sentimiento"] = df["sentimiento"].str.strip()

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363 entries, 0 to 362
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   titulo       363 non-null    object
 1   cuerpo       363 non-null    object
 2   sentimiento  363 non-null    object
 3   texto        363 non-null    object
dtypes: object(4)
memory usage: 11.5+ KB


In [13]:
df["sentimiento"].value_counts()

sentimiento
NEUTRAL     189
NEGATIVO    104
POSITIVO     70
Name: count, dtype: int64

<hr/>

In [14]:
import re
import nltk
import spacy
from nltk.corpus import stopwords

In [15]:
!pip install https://github.com/explosion/spacy-models/releases/download/es_core_news_lg-3.8.0/es_core_news_lg-3.8.0-py3-none-any.whl

     ━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/568.0 MB 5.5 MB/s eta 0:01:25
ERROR: Wheel 'es-core-news-lg' located at /tmp/pip-unpack-2n1ei8b9/es_core_news_lg-3.8.0-py3-none-any.whl is invalid.

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [16]:
!python -m spacy validate

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.8.7) =================
ℹ spaCy installation: /usr/local/lib/python3.10/site-packages/spacy

NAME              SPACY            VERSION                            
es_core_news_lg   >=3.8.0,<3.9.0   3.8.0   ✔



In [17]:
nltk.download("stopwords")
stopwords_es = set(stopwords.words("spanish"))
nlp = spacy.load("es_core_news_lg")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
def clean_text(text):
    # 1. Eliminar menciones, URLs y caracteres especiales innecesarios
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)  # @usuario
    text = re.sub(r'https?://\S+', '', text)  # URLs
    text = re.sub(r'\n', ' ', text)  # Saltos de línea
    text = re.sub(r'[^A-Za-zÁÉÍÓÚÑáéíóúñ0-9\s\$]', '', text)  # Elimina emojis y símbolos
    
    # 2. Pasar a minúsculas
    text = text.lower()
    
    # 3. Tokenizar con spaCy
    doc = nlp(text)
    
    # 4. Filtrar tokens
    tokens = [
        token.text
        for token in doc
        if token.text.lower() not in stopwords_es  # eliminar stopwords
        and not token.is_punct
        and not token.is_space
    ]
    
    return " ".join(tokens)

# Aplicar al DataFrame
df["texto_limpio"] = df["texto"].apply(clean_text)

In [19]:
pd.set_option("display.max_colwidth", None)

In [20]:
df[["texto", "texto_limpio"]].head(7)

,texto,texto_limpio
0,"Trading en Tiktok Que onda derrepente cada vairo que ofrece su curso, antes co el novio de la onichan nms era pesado pero ahora se reproducieron Hasta ese man que hacía tiktok de macanadas en el mercado 4 co hace TRADING, después esta ese Aquino que con un cripto cada tanto gua’u efectivizar y para colmo tiene gente boba que comenta toda la hora brooo líder espectacular quiero entrar al curso🙏🙏 tenes que irte de viaje también a dubai vos líder 🙏🙏🙏, se están volviendo peor que los colorados",trading tiktok onda derrepente cada vairo ofrece curso co novio onichan nms pesado ahora reproducieron man hacía tiktok macanadas mercado 4 co hace trading después aquino cripto cada guau efectivizar colmo gente boba comenta toda hora brooo líder espectacular quiero entrar curso tenes irte viaje dubai vos líder volviendo peor colorados
1,"Percibir salario desde trabajo remoto Hola, el próximo mes empezaré a trabajar remoto para una multinacional extranjera. Puedo cobrar en USD o en cripto... Ustedes que todo lo saben... ¿Cuál es la mejor opción?\nSi es en USD, ¿qué bancos recomiendan y por qué? \nSi es en cripto, ¿qué billeteras recomiendan y por qué?\n ¿Sería buena idea cobrar una parte en USD y el resto en cripto? Compartan su experiencia /tips",percibir salario trabajo remoto hola próximo mes empezaré trabajar remoto multinacional extranjera puedo cobrar usd cripto ustedes saben cuál mejor opción si usd bancos recomiendan si cripto billeteras recomiendan buena idea cobrar parte usd resto cripto compartan experiencia tips
2,Apple gift cards Tengo 400$ en Apple gift cards y quiero vender. Ya sea transferencia o cripto. \nVendo 100$ de gift cards por 95$,apple gift cards 400 $ apple gift cards quiero vender transferencia cripto vendo 100 $ gift cards 95 $
3,"Recibir pagos del exterior sin tener problemas con Set Como paraguayo cual es la manera más simple de cobrar por trabajos que sea compatible freelancer y fiverr? Sin tener que hacer jugadas de convertir la plata a cripto, luego a euros y luego a guaraníes?",recibir pagos exterior tener problemas set paraguayo manera simple cobrar trabajos compatible freelancer fiverr tener hacer jugadas convertir plata cripto luego euros luego guaraníes
4,"Tarjetas de credito cobrando en cripto Alguien sabe si hay alguna forma de que ciertos bancos me habiliten tarjetas de credito si cobro en cripto? Trabajo para una empresa de afuera y cobro en dolares, para ahorrarme gastos cobro en cripto pero ulltimamente estoy queriendo quitar tarjetas de credito y no se como poder blanquear eso, consejos?",tarjetas credito cobrando cripto alguien sabe si alguna forma ciertos bancos habiliten tarjetas credito si cobro cripto trabajo empresa afuera cobro dolares ahorrarme gastos cobro cripto ulltimamente queriendo quitar tarjetas credito poder blanquear consejos
5,"Estaría interesante conocer personas que esten metidas en el mundo cripto y el trading para intercambiar palabras, escriban!! (sin texto)",interesante conocer personas esten metidas mundo cripto trading intercambiar palabras escriban texto
6,"Hay alguna forma práctica de cambiar cripto a Gs. para no residentes? O sea, para alguien que no tenga cuenta bancaria allá. \n\nHay algún método confiable y que no te maten con la comisión?",alguna forma práctica cambiar cripto gs residentes alguien cuenta bancaria allá algún método confiable maten comisión


In [21]:
df[["texto", "texto_limpio"]].tail(7)

,texto,texto_limpio
356,El ecosistema de Bitcoin crece en Paraguay impulsado por la minería y la educación.,ecosistema bitcoin crece paraguay impulsado minería educación
357,"“Fue una conversación inspiradora entre personas con trayectorias, contextos y experiencias distintas, reunidas en Asunción por primera vez con el propósito común de pensar, construir y mejorar Bitcoin”, dijo Juan Pessolani, voluntario de Bitcoin Paraguay.",conversación inspiradora personas trayectorias contextos experiencias distintas reunidas asunción primera vez propósito común pensar construir mejorar bitcoin dijo juan pessolani voluntario bitcoin paraguay
358,"Paraguay tiene todo para ser un hub tecnológico en América Latina, según Cofundadora de X4T\n\n📌X4T nació para resolver una necesidad concreta: facilitar el acceso a criptomonedas desde Paraguay. Con una plataforma 100% local, apuesta por la educación financiera y la inclusión digital.\n\n👉La empresa nace a partir de dos necesidades que detectamos: la falta de un exchange local y la necesidad de acercar el mundo cripto a las personas, explicó Nicole Cobelli, Vicepresidente y Cofundadora de la plataforma durante el evento Mujeres Power Summit.",paraguay ser hub tecnológico américa latina según cofundadora x4 t x4 t nació resolver necesidad concreta facilitar acceso criptomonedas paraguay plataforma 100 local apuesta educación financiera inclusión digital empresa nace partir dos necesidades detectamos falta exchange local necesidad acercar mundo cripto personas explicó nicole cobelli vicepresidente cofundadora plataforma evento mujeres power summit
359,"¡Qué lindo que va a estar esto 🇵🇾! \n\nParaguay Blockchain Summit. \n\n@criptoboi\n es un gran builder de la región y me alegra ver el crecimiento que tanto el, su equipo y la comunidad paraguaya están logrando para acelerar la adopción de criptomonedas.",lindo va paraguay blockchain summit gran builder región alegra ver crecimiento equipo comunidad paraguaya logrando acelerar adopción criptomonedas
360,"Desde hace años, en @cripto_py venimos desarrollando el mercado cripto en Paraguay. El Paraguay Blockchain Summit @PBSpy_\n representa el resultado de años de esfuerzo.\n\nCriptoPy se une a la Cámara Paraguaya de Fintech con una misión clara: convertir a Asunción en la capital blockchain de Latinoamérica.\n\nRegulación, medios de pago, educación y tokenización, todo en un solo lugar.\n\nSpeakers internacionales, reguladores locales y comunidades globales reunidos por una sola causa: el futuro descentralizado.",hace años venimos desarrollando mercado cripto paraguay paraguay blockchain summit representa resultado años esfuerzo criptopy une cámara paraguaya fintech misión clara convertir asunción capital blockchain latinoamérica regulación medios pago educación tokenización solo lugar speakers internacionales reguladores locales comunidades globales reunidos sola causa futuro descentralizado
361,"🚨 RESUMEN DE LAS ÚLTIMAS 8 HORAS 👇\n\n🌍 Paraguay se está convirtiendo en un centro importante para Bitcoin con grandes inversiones en minería.\n\n💡 Programas educativos como ""Mi Primer Bitcoin"" impulsan el conocimiento financiero en criptomonedas.\n\n📈 Ventas de XRP en Binance muestran una fuerte demanda, sugiriendo una tendencia alcista próximamente.\n\n🏦 Standard Chartered se une a la ola de adopción de Bitcoin, ofreciendo nuevas opciones de inversión cripto.\n\n🔗 La actualización Pectra impulsa la actividad de Ethereum con mayor uso de aplicaciones descentralizadas.\n\n💶 El Banco Central Europeo explora la implementación del euro digital para mejorar la eficiencia financiera.",resumen últimas 8 horas paraguay convirtiendo centro importante bitcoin grandes inversiones minería programas educativos primer bitcoin impulsan conocimiento financiero criptomonedas ventas xrp binance muestran fuerte demanda sugiriendo tendencia alcista próximamente standard chartered une ola adopción bitcoin ofreciendo nuevas opciones inversión cripto actualización pectra impul

<hr/>

In [22]:
# Importar las librerías necesarias
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

In [38]:
sentimiento_mapping = {
    "POSITIVO": 0,
    "NEGATIVO": 1,
    "NEUTRAL": 2
}

df["valor_sentimiento"] = df["sentimiento"].map(sentimiento_mapping)

In [39]:
# Separar features y target
X = df["texto_limpio"]
y = df["valor_sentimiento"]

# Dividir el conjunto de datos en train y test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [68]:
# Vectorizar con TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [69]:
# Entrenar modelo
model = LogisticRegression(class_weight="balanced", random_state=42)
model.fit(X_train_tfidf, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,42
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [70]:
# Predecir y evaluar
y_pred = model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.6712328767123288
              precision    recall  f1-score   support

           0       0.45      0.36      0.40        14
           1       0.63      0.81      0.71        21
           2       0.77      0.71      0.74        38

    accuracy                           0.67        73
   macro avg       0.62      0.63      0.62        73
weighted avg       0.67      0.67      0.67        73



<hr/>

In [61]:
from sklearn.ensemble import RandomForestClassifier

# Entrenar modelo
model = RandomForestClassifier(class_weight="balanced", random_state=42)
model.fit(X_train_tfidf, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [62]:
# Predecir y evaluar
y_pred = model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.6027397260273972
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.75      0.29      0.41        21
           2       0.58      1.00      0.74        38

    accuracy                           0.60        73
   macro avg       0.44      0.43      0.38        73
weighted avg       0.52      0.60      0.50        73



/usr/local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


<hr/>

In [63]:
from sklearn.svm import LinearSVC

# Crear y entrenar modelo SVM
model = LinearSVC(class_weight="balanced", random_state=42)
model.fit(X_train_tfidf, y_train)

,penalty,'l2'
,loss,'squared_hinge'
,dual,'auto'
,tol,0.0001
,C,1.0
,multi_class,'ovr'
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,verbose,0
,random_state,42


In [52]:
# Predecir y evaluar
y_pred = model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.6712328767123288
              precision    recall  f1-score   support

           0       0.71      0.36      0.48        14
           1       0.57      0.76      0.65        21
           2       0.74      0.74      0.74        38

    accuracy                           0.67        73
   macro avg       0.67      0.62      0.62        73
weighted avg       0.68      0.67      0.66        73



<hr/>